# Import dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
csv_path = '/content/drive/MyDrive/Notes/WOA7015 - Advanced Machine Learning/datasets/diabetes_012_health_indicators_BRFSS2015.csv'

# csv_path = 'PATH NAME HERE'

Downsampling

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, balanced_accuracy_score


df = pd.read_csv(csv_path)

# extract last 15% of original dataset as test dataset
test_start_index = len(df) // 100 * 85
train_df = df.iloc[:test_start_index]
test_df = df.iloc[test_start_index + 1:]

min_class_count = train_df['Diabetes_012'].value_counts().min()
class_groups = [train_df[train_df['Diabetes_012'] == c] for c in train_df['Diabetes_012'].unique()]
downsampled_classes = [group.sample(n=min_class_count, random_state=42)
                        for group in class_groups]

train_df = pd.concat(downsampled_classes)

train_features = train_df.drop(columns=["Diabetes_012"])
train_labels = train_df["Diabetes_012"]

test_features = test_df.drop(columns=["Diabetes_012"])
test_labels = test_df["Diabetes_012"]

In [ ]:

count_class = train_labels.value_counts() # Count the occurrences of each class
plt.bar(count_class.index, count_class.values)
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Class Distribution')
plt.show()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# data (as pandas dataframes)
X_train, X_val, y_train, y_val = train_test_split(
    train_features, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

class DiabetesDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features.values, dtype=torch.float32).to(device) 
        self.targets = torch.tensor(targets.values, dtype=torch.long).to(device)    

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

# Create Dataset
train_dataset = DiabetesDataset(X_train, y_train)
val_dataset = DiabetesDataset(X_val, y_val)
test_dataset = DiabetesDataset(test_features, test_labels)

Model code starting here

Use `train_dataset`, `val_dataset`, `test_dataset` as the input dataset

Train Dataset has

| Class | Count |
| --- | --- |
| 0.0 | 3978|
| 2.0 | 3978|
| 1.0 | 3978|

Test Dataset has

| Class | Count |
| --- | --- |
| 0.0 | 32427|
| 2.0 | 5039|
| 1.0 | 653|

Example below:

In [4]:
batch_size = 128

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# MLNN 1

In [6]:

# Define the neural network
class Net(nn.Module):
    def __init__(self, input_size, num_classes):
        super(Net, self).__init__()
        self.input_layer = nn.Linear(input_size, 32)
        self.h1 = nn.Linear(32, 1024)
        self.h2 = nn.Linear(1024, 64)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = F.relu(self.input_layer(x))
        x = F.relu(self.h1(x))
        x = F.relu(self.h2(x))
        x = self.logsoftmax(self.output_layer(x))
        return x

In [ ]:
# Initialize the model, loss function, and optimizer
input_size = len(X_train.columns)  # Number of input features
num_classes = len(train_labels.unique())
model = Net(input_size, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
# criterion = nn.MultiMarginLoss(margin=0.1)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 100
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    if epoch % 10 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

# Inference on the test set
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
  for inputs, labels in test_loader:
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    predictions.extend(predicted.tolist())
    true_labels.extend(labels.tolist())

In [ ]:
# Plotting the results
plt.figure(figsize=(6, 3))
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

accuracy = accuracy_score(true_labels, predictions)
balanced_accuracy = balanced_accuracy_score(true_labels, predictions)
f1_score = f1_score(true_labels, predictions, average='weighted')
print(f'Test accuracy: {accuracy:.6f}')
print(f'Balanced Test accuracy: {balanced_accuracy:.6f}')
print(f'F1 score: {f1_score:.6f}')
print(f'{accuracy:.6f}')
print(f'{balanced_accuracy:.6f}')
print(f'{f1_score:.6f}')

cm = confusion_matrix(true_labels, predictions)
plt.figure(figsize=(4, 3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

# MLNN 2

In [ ]:

# Define the neural network
class Net(nn.Module):
    def __init__(self, input_size, num_classes):
        super(Net, self).__init__()
        self.input_layer = nn.Linear(input_size, 128)
        self.h1 = nn.Linear(128, 512)
        self.h2 = nn.Linear(512, 1024)
        self.h3 = nn.Linear(1024, 128)
        self.output_layer = nn.Linear(128, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.tanh(self.input_layer(x))
        x = F.tanh(self.h1(x))
        x = F.tanh(self.h2(x))
        x = F.tanh(self.h3(x))
        x = self.softmax(self.output_layer(x))
        return x

In [ ]:
# Initialize the model, loss function, and optimizer
input_size = len(X_train.columns)  # Number of input features
num_classes = len(train_labels.unique())
model = Net(input_size, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
# criterion = nn.MultiMarginLoss(margin=0.1)
optimizer = optim.Adam(model.parameters(), lr=0.003, weight_decay=1e-4)
scheduler = ExponentialLR(optimizer, gamma=0.9)

# Training loop
num_epochs = 300
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)

    if epoch % 10 == 0:
      scheduler.step()

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    if epoch % 10 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

# Inference on the test set
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
  for inputs, labels in test_loader:
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    predictions.extend(predicted.tolist())
    true_labels.extend(labels.tolist())

In [ ]:
# Plotting the results
plt.figure(figsize=(6, 3))
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

accuracy = accuracy_score(true_labels, predictions)
balanced_accuracy = balanced_accuracy_score(true_labels, predictions)
f1_score = f1_score(true_labels, predictions, average='weighted')
print(f'Test accuracy: {accuracy:.6f}')
print(f'Balanced Test accuracy: {balanced_accuracy:.6f}')
print(f'F1 score: {f1_score:.6f}')
print(f'{accuracy:.6f}')
print(f'{balanced_accuracy:.6f}')
print(f'{f1_score:.6f}')

cm = confusion_matrix(true_labels, predictions)
plt.figure(figsize=(4, 3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

# MLNN 3

In [ ]:

# Define the neural network
class Net(nn.Module):
    def __init__(self, input_size, num_classes):
        super(Net, self).__init__()
        self.input_layer = nn.Linear(input_size, 128)
        self.bn0 = nn.BatchNorm1d(128)
        self.h1 = nn.Linear(128, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.h2 = nn.Linear(1024, 5120)
        self.bn2 = nn.BatchNorm1d(5120)
        self.h3 = nn.Linear(5120, 1024)
        self.bn3 = nn.BatchNorm1d(1024)
        self.h4 = nn.Linear(1024, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.output_layer = nn.Linear(64, num_classes)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = F.sigmoid(self.input_layer(x))
        x = self.bn0(x)
        x = F.sigmoid(self.h1(x))
        x = self.bn1(x)
        x = F.sigmoid(self.h2(x))
        x = self.bn2(x)
        x = F.sigmoid(self.h3(x))
        x = self.bn3(x)
        x = F.sigmoid(self.h4(x))
        x = self.bn4(x)
        x = self.logsoftmax(self.output_layer(x))
        return x

In [ ]:
# Initialize the model, loss function, and optimizer
input_size = len(X_train.columns)  # Number of input features
num_classes = len(train_labels.unique())
model = Net(input_size, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
# criterion = nn.MultiMarginLoss(margin=0.1)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)
scheduler = ExponentialLR(optimizer, gamma=0.9)

# Training loop
num_epochs = 200
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)

    if epoch % 10 == 0:
      scheduler.step()

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    if epoch % 10 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

# Inference on the test set
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
  for inputs, labels in test_loader:
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    predictions.extend(predicted.tolist())
    true_labels.extend(labels.tolist())

In [ ]:
# Plotting the results
plt.figure(figsize=(6, 3))
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

accuracy = accuracy_score(true_labels, predictions)
balanced_accuracy = balanced_accuracy_score(true_labels, predictions)
f1_score = f1_score(true_labels, predictions, average='weighted')
print(f'Test accuracy: {accuracy:.6f}')
print(f'Balanced Test accuracy: {balanced_accuracy:.6f}')
print(f'F1 score: {f1_score:.6f}')
print(f'{accuracy:.6f}')
print(f'{balanced_accuracy:.6f}')
print(f'{f1_score:.6f}')

cm = confusion_matrix(true_labels, predictions)
plt.figure(figsize=(4, 3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

# MLNN 4

In [ ]:

# Define the neural network
class Net(nn.Module):
    def __init__(self, input_size, num_classes):
        super(Net, self).__init__()
        self.input_layer = nn.Linear(input_size, 128)
        self.h1 = nn.Linear(128, 512)
        self.h2 = nn.Linear(512, 1024)
        self.h3 = nn.Linear(1024, 128)
        self.output_layer = nn.Linear(128, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.tanh(self.input_layer(x))
        x = F.tanh(self.h1(x))
        x = F.tanh(self.h2(x))
        x = F.tanh(self.h3(x))
        x = self.softmax(self.output_layer(x))
        return x

In [ ]:
# Initialize the model, loss function, and optimizer
input_size = len(X_train.columns)  # Number of input features
num_classes = len(train_labels.unique())
model = Net(input_size, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
# criterion = nn.MultiMarginLoss(margin=0.1)
optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-4)
scheduler = ExponentialLR(optimizer, gamma=0.9)

# Training loop
num_epochs = 300
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)

    if epoch % 10 == 0:
      scheduler.step()

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    if epoch % 10 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

# Inference on the test set
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
  for inputs, labels in test_loader:
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    predictions.extend(predicted.tolist())
    true_labels.extend(labels.tolist())

In [ ]:
# Plotting the results
plt.figure(figsize=(6, 3))
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

accuracy = accuracy_score(true_labels, predictions)
balanced_accuracy = balanced_accuracy_score(true_labels, predictions)
f1_score = f1_score(true_labels, predictions, average='weighted')
print(f'Test accuracy: {accuracy:.6f}')
print(f'Balanced Test accuracy: {balanced_accuracy:.6f}')
print(f'F1 score: {f1_score:.6f}')
print(f'{accuracy:.6f}')
print(f'{balanced_accuracy:.6f}')
print(f'{f1_score:.6f}')

cm = confusion_matrix(true_labels, predictions)
plt.figure(figsize=(4, 3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()